In [6]:
import numpy as np
import plotly.graph_objs as go
import plotly.subplots as sp
from scipy.signal import butter, lfilter, freqz, iirnotch
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox
import ipywidgets as widgets

np.random.seed(0)

t = np.linspace(0, 1.0, 1000, endpoint=False)
low_freq = np.sin(2 * np.pi * 5 * t)
mid_freq = np.sin(2 * np.pi * 50 * t)
high_freq = np.sin(2 * np.pi * 200 * t)
signal = low_freq + mid_freq + high_freq
noise = np.random.normal(size=t.shape)


def butter_filter(data, lowcut=None, highcut=None, fs=1000, order=4, btype='low'):
    nyq = 0.5 * fs
    if btype == 'band':
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype=btype)
    elif btype == 'low':
        high = highcut / nyq
        b, a = butter(order, high, btype=btype)
    elif btype == 'high':
        low = lowcut / nyq
        b, a = butter(order, low, btype=btype)
    else:
        raise ValueError("Invalid filter type")
    return lfilter(b, a, data), b, a


def compute_gain(b, a):
    w, h = freqz(b, a, worN=8000)
    f = 0.5 * 1000 * w / np.pi
    gain = 20 * np.log10(np.maximum(np.abs(h), 1e-10))
    return f, gain


def update(lowcut=10.0, highcut=100.0, notch_freq=50.0, sample_rate=25, amp_factor=2.0, noise_level=0.5,
           show_amp=True, show_lp=False, show_hp=False, show_bp=False, show_notch=False,
           show_sampling=False, show_sampling_bp=False):

    noisy_signal = signal + noise_level * noise
    amplified_signal = amp_factor * noisy_signal
    band_passed, b_bp, a_bp = butter_filter(noisy_signal, lowcut=highcut, highcut=lowcut, btype='band')

    sections = ["original_noisy"]
    if show_amp:
        sections.append("amp")
    if show_lp:
        sections.append("lp")
    if show_hp:
        sections.append("hp")
    if show_bp:
        sections.append("bp")
    if show_notch:
        sections.append("notch")
    if show_sampling:
        sections.append("sampling")
    if show_sampling_bp:
        sections.append("sampling_filtered")

    fig = sp.make_subplots(rows=len(sections), cols=2, vertical_spacing=0.15,
                           subplot_titles=[
                               "Original Signal (5Hz + 50Hz + 200Hz)", "Noisy Signal",
                               *(title for sec in sections[1:] for title in ([f"{sec.capitalize()} Output", f"{sec.capitalize()} Response"] if not (sec.startswith("sampling") or sec.startswith("amp")) else [f"{sec.replace('_', ' ').capitalize()}", ""]))
                           ])

    row_idx = 1
    fig.add_trace(go.Scatter(x=t, y=signal, name='Original'), row=row_idx, col=1)
    fig.add_trace(go.Scatter(x=t, y=noisy_signal, name='Noisy'), row=row_idx, col=2)
    row_idx += 1

    if show_amp:
        fig.add_trace(go.Scatter(x=t, y=amplified_signal, name='Amplified'), row=row_idx, col=1)
        row_idx += 1

    if show_lp:
        low_passed, b_lp, a_lp = butter_filter(noisy_signal, highcut=lowcut, btype='low')
        f_lp, gain_lp = compute_gain(b_lp, a_lp)
        fig.add_trace(go.Scatter(x=t, y=low_passed, name='Low-Pass'), row=row_idx, col=1)
        fig.add_trace(go.Scatter(x=f_lp, y=gain_lp, name='Low-Pass Response'), row=row_idx, col=2)
        fig.update_xaxes(type='log', row=row_idx, col=2)
        row_idx += 1

    if show_hp:
        high_passed, b_hp, a_hp = butter_filter(noisy_signal, lowcut=highcut, btype='high')
        f_hp, gain_hp = compute_gain(b_hp, a_hp)
        fig.add_trace(go.Scatter(x=t, y=high_passed, name='High-Pass'), row=row_idx, col=1)
        fig.add_trace(go.Scatter(x=f_hp, y=gain_hp, name='High-Pass Response'), row=row_idx, col=2)
        fig.update_xaxes(type='log', row=row_idx, col=2)
        row_idx += 1

    if show_bp:
        f_bp, gain_bp = compute_gain(b_bp, a_bp)
        fig.add_trace(go.Scatter(x=t, y=band_passed, name='Band-Pass'), row=row_idx, col=1)
        fig.add_trace(go.Scatter(x=f_bp, y=gain_bp, name='Band-Pass Response'), row=row_idx, col=2)
        fig.update_xaxes(type='log', row=row_idx, col=2)
        row_idx += 1

    if show_notch:
        b_notch, a_notch = iirnotch(notch_freq, 30, fs=1000)
        notch_filtered = lfilter(b_notch, a_notch, noisy_signal)
        f_notch, gain_notch = compute_gain(b_notch, a_notch)
        fig.add_trace(go.Scatter(x=t, y=notch_filtered, name='Notch-Filtered'), row=row_idx, col=1)
        fig.add_trace(go.Scatter(x=f_notch, y=gain_notch, name='Notch Response'), row=row_idx, col=2)
        fig.update_xaxes(type='log', row=row_idx, col=2)
        row_idx += 1

    if show_sampling:
        sampled_time = np.linspace(0, 1, sample_rate, endpoint=False)
        sampled_signal = np.interp(sampled_time, t, noisy_signal)
        quantized_signal = np.round(sampled_signal * 10) / 10
        fig.add_trace(go.Scatter(x=t, y=noisy_signal, name='Noisy Input'), row=row_idx, col=1)
        fig.add_trace(go.Scatter(x=sampled_time, y=sampled_signal, mode='markers+lines', name='Sampled'), row=row_idx, col=1)
        fig.add_trace(go.Scatter(x=sampled_time, y=quantized_signal, mode='markers+lines', name='Quantized'), row=row_idx, col=2)
        row_idx += 1

    if show_sampling_bp:
        sampled_time = np.linspace(0, 1, sample_rate, endpoint=False)
        sampled_signal = np.interp(sampled_time, t, band_passed)
        quantized_signal = np.round(sampled_signal * 10) / 10
        fig.add_trace(go.Scatter(x=t, y=band_passed, name='Bandpassed Input'), row=row_idx, col=1)
        fig.add_trace(go.Scatter(x=sampled_time, y=sampled_signal, mode='markers+lines', name='Sampled (BP)'), row=row_idx, col=1)
        fig.add_trace(go.Scatter(x=sampled_time, y=quantized_signal, mode='markers+lines', name='Quantized (BP)'), row=row_idx, col=2)
        row_idx += 1

    for r in range(1, len(sections)+1):
        for c in range(1, 3):
            if c == 1:
                fig.update_xaxes(title_text='Time (s)', row=r, col=c, type='linear')
                fig.update_yaxes(title_text='Amplitude', row=r, col=c)
            else:
                if any(resp in sections[r-1] for resp in ["lp", "hp", "bp", "notch"]):
                    fig.update_xaxes(title_text='Frequency (Hz)', type='log', row=r, col=c)
                    fig.update_yaxes(title_text='Gain (dB)', row=r, col=c)
                else:
                    fig.update_xaxes(title_text='Time (s)', type='linear', row=r, col=c)
                    fig.update_yaxes(title_text='Amplitude', row=r, col=c)

    fig.update_layout(height=len(sections) * 320, width=1000, title_text="Signal Processing Concepts (Interactive)", showlegend=False)
    fig.show()

interact(update,
         lowcut=FloatSlider(min=0.5, max=300.0, step=0.1, value=100.0, description='Low-Pass'),
         highcut=FloatSlider(min=0.1, max=300.0, step=0.1, value=10.0, description='High-Pass'),
         notch_freq=FloatSlider(min=1.0, max=300.0, step=1.0, value=50.0, description='Notch Freq'),
         sample_rate=IntSlider(min=5, max=100, step=1, value=25, description='Sample Rate'),
         amp_factor=FloatSlider(min=1, max=15.0, step=0.1, value=2.0, description='Amplification'),
         noise_level=FloatSlider(min=0.0, max=2.0, step=0.1, value=0.5, description='Noise Level'),
         show_amp=Checkbox(value=False, description='Amplification'),
         show_lp=Checkbox(value=False, description='Lowpass'),
         show_hp=Checkbox(value=False, description='Highpass'),
         show_bp=Checkbox(value=False, description='Bandpass'),
         show_notch=Checkbox(value=False, description='Notch'),
         show_sampling=Checkbox(value=False, description='Sampling'),
         show_sampling_bp=Checkbox(value=False, description='Sampling (Bandpass)'))


interactive(children=(FloatSlider(value=100.0, description='Low-Pass', max=300.0, min=0.5), FloatSlider(value=…

<function __main__.update(lowcut=10.0, highcut=100.0, notch_freq=50.0, sample_rate=25, amp_factor=2.0, noise_level=0.5, show_amp=True, show_lp=False, show_hp=False, show_bp=False, show_notch=False, show_sampling=False, show_sampling_bp=False)>